# Perception Block

In [1]:
import os
import importlib
import sys

import numpy as np
from pydrake.all import (
    AbstractValue,
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    PiecewisePose,
    ProcessModelDirectives,
    RigidTransform,
    RotationMatrix,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    LeafSystem,
    ConstantVectorSource
)
from IPython.display import display, SVG
import pydot
import matplotlib.pyplot as plt
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import JointStiffnessController
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser
from manipulation.systems import AddIiwaDifferentialIK

sys.path.append('.')
import env_ingredient_add
importlib.reload(env_ingredient_add)

import

<module 'env_ingredient_add' from '/root/work/manipulation/project/env_ingredient_add.py'>

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://92fbdad2-a4a9-4ef8-b493-c307060b34a1.deepnoteproject.com/7001/


# Sim environment  - Table  + top down camera

In [3]:
def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=xmls
    )
    simulator = Simulator(station)
    context = simulator.get_mutable_context()
    x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
    station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()

if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
    #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
    scene = env_ingredient_add.pizza_state_environment_set_up()
    xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
else:
    #scene = open("objects/environment_setup.yaml") # local setup
    scene = env_ingredient_add.pizza_state_environment_set_up()
    xmls = [os.getcwd() + "/package.xml"]

scenario = LoadScenario(data=scene)

preview_scene_given_directives(scene)



## Perception - Plumbing 

In [5]:
class CameraSystem:
    def __init__(self, idx, meshcat, diagram, context):
        self.idx = idx

        # Read images
        depth_im_read = (
            diagram.GetOutputPort("camera{}_depth_image".format(idx))
            .Eval(context)
            .data.squeeze()
        )
        self.depth_im = deepcopy(depth_im_read)
        self.depth_im[self.depth_im == np.inf] = 10.0
        self.rgb_im = (
            diagram.GetOutputPort("camera{}_rgb_image".format(idx)).Eval(context).data
        )

        # Visualize
        point_cloud = diagram.GetOutputPort("camera{}_point_cloud".format(idx)).Eval(
            context
        )
        meshcat.SetObject(f"Camera {idx} point cloud", point_cloud)

        # Get other info about the camera
        cam = diagram.GetSubsystemByName("camera" + str(idx))
        cam_context = cam.GetMyMutableContextFromRoot(context)
        self.X_WC = cam.body_pose_in_world_output_port().Eval(cam_context)
        self.cam_info = cam.default_depth_render_camera().core().intrinsics()

    def project_depth_to_pC(self, depth_pixel):
        """
        project depth pixels to points in camera frame
        using pinhole camera model
        Input:
            depth_pixels: numpy array of (nx3) or (3,)
        Output:
            pC: 3D point in camera frame, numpy array of (nx3)
        """
        # switch u,v due to python convention
        v = depth_pixel[:, 0]
        u = depth_pixel[:, 1]
        Z = depth_pixel[:, 2]
        cx = self.cam_info.center_x()
        cy = self.cam_info.center_y()
        fx = self.cam_info.focal_x()
        fy = self.cam_info.focal_y()
        X = (u - cx) * Z / fx
        Y = (v - cy) * Z / fy
        pC = np.c_[X, Y, Z]
        return pC

bot = PizzaBot(scenario)
environment_diagram, environment_context = bot.diagram, bot.context
# environment_context = ClutteredSceneSystem()
cameras = []
cameras.append(CameraSystem(0, meshcat, environment_diagram, environment_context))


NameError: name 'PizzaBot' is not defined

In [ ]:
plt.imshow(cameras[0].rgb_im)
plt.title("View from camera 0")
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>